# Prompts

A prompt is the input text or instruction given to a language model to generate a response.

- Setup: 
This is the setup to create a ChatLLM model using Langchain, please refer to model and integration to see the other possible options

In [39]:
# =================== Section to change according to your choice of APIs you have access to===============
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

## PromptTemplates

- Basic prompt template


In [27]:
from langchain_core.prompts import PromptTemplate
template = "Tell me a {adjective} joke about {subject}."
prompt = PromptTemplate(
    input_variables=["adjective", "subject"],  # can be commented
    template=template
)

formatted_prompt = prompt.format(adjective="funny", subject="programming")
print(formatted_prompt)

Tell me a funny joke about programming.


- Few-shot prompting

In [28]:
few_shot_template = """
I'm going to give you some examples of analyzing sentiment:

Text: I love this product!
Sentiment: Positive

Text: This is the worst experience ever.
Sentiment: Negative

Text: {input_text}
Sentiment:"""

few_shot_prompt = PromptTemplate(
    input_variables=["input_text"], # can be commented
    template=few_shot_template
)

formatted_few_shot_prompt = few_shot_prompt.format(input_text="I am feeling blue today")
print(formatted_few_shot_prompt)



I'm going to give you some examples of analyzing sentiment:

Text: I love this product!
Sentiment: Positive

Text: This is the worst experience ever.
Sentiment: Negative

Text: I am feeling blue today
Sentiment:


## Chat Prompt Templates

PromptTemplate: Used for standard, non-chat-based prompts where you pass a single string of text.

ChatPromptTemplate: Specifically for chat-based models, where you manage multiple message types (like user and assistant) in a conversation.

In [30]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant specialized in {topic}."),
    ("human", "Can you help me with: {question}?")
])

messages = chat_template.format_messages(
    topic="data science",
    question="How do I analyze time series data?"
)
print(messages)

[SystemMessage(content='You are a helpful AI assistant specialized in data science.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Can you help me with: How do I analyze time series data??', additional_kwargs={}, response_metadata={})]


- Using LangChain Expression Language (LCEL)

LangChain Expression Language (LCEL) is a specialized language in LangChain that allows you to define and execute dynamic expressions within prompts, enabling flexible and customized interactions with language models. It helps in interpolating variables and processing logic within templates.

In [32]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Write a poem about {topic}")
output_parser = StrOutputParser()

chain = prompt | model | output_parser
try:
    response = chain.invoke({"topic": "autumn leaves"})
except Exception as e:
    print(e)

Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


## Output Parsers
In LangChain, an Output Parser is a component that takes the raw output from an LLM (usually a string) and converts it into a structured format—such as a Python dictionary, list, Pydantic model, or even LangChain-specific objects.

In [38]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

# Define Movie and MovieList Pydantic models
class Movie(BaseModel):
    title: str = Field(description="Title of the movie")
    director: str = Field(description="Director of the movie")
    year: int = Field(description="Year the movie was released")

class MovieList(BaseModel):
    movies: List[Movie] = Field(description="List of movies")

# Create the prompt template
prompt = ChatPromptTemplate.from_template(
    """Generate {number} movies about {topic}. List their titles, directors, and release years.
    """
)

# Create the parser
parser = PydanticOutputParser(pydantic_object=MovieList)

# Create the chain with the model and parser
chain = prompt | model | parser

# Invoke the chain to get movie information
try:
    movies = chain.invoke({
        "number": 3,
        "topic": "time travel",
    })
    
    # Output the movies
    print(movies)
except Exception as e:
    print(e)


Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
